In [6]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# EDA for Sierra Leone Solar Farm Data"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": None,
   "metadata": {},
   "outputs": [],
   "source": [
    "import pandas as pd\n",
    "import numpy as np\n",
    "import matplotlib.pyplot as plt\n",
    "import seaborn as sns\n",
    "from scipy.stats import zscore\n",
    "\n",
    "sns.set_style(\"whitegrid\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 1. Setup and Data Loading"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": None,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Load the Sierra Leone dataset\n",
    "# Ensure the 'data' directory is created and 'sierraleone-bumbuna.csv' is placed inside it.\n",
    "try:\n",
    "    df_sierraleone = pd.read_csv('../data/sierraleone-bumbuna.csv', parse_dates=['Timestamp (yyyy-mm-dd hh:mm)'])\n",
    "    df_sierraleone = df_sierraleone.rename(columns={'Timestamp (yyyy-mm-dd hh:mm)': 'Timestamp'})\n",
    "    print(\"Sierra Leone data loaded successfully.\")\n",
    "except FileNotFoundError:\n",
    "    print(\"Error: 'sierraleone-bumbuna.csv' not found. Please ensure it's in the '../data/' directory.\")\n",
    "    # Create a dummy DataFrame for demonstration if the file is not found\n",
    "    print(\"Creating dummy data for demonstration purposes.\")\n",
    "    data = {\n",
    "        'Timestamp': pd.to_datetime(['2025-01-01 00:00', '2025-01-01 01:00', '2025-01-01 02:00', '2025-01-01 03:00', '2025-01-01 04:00']),\n",
    "        'GHI (W/m²)': [0.5, 0.8, 1.2, 15, 25],\n",
    "        'DNI (W/m²)': [0, 0, 0, 7, 18],\n",
    "        'DHI (W/m²)': [0.5, 0.8, 1.2, 8, 7],\n",
    "        'ModA (W/m²)': [0, 0, 0, 10, 20],\n",
    "        'ModB (W/m²)': [0, 0, 0, 9, 19],\n",
    "        'Tamb (°C)': [26, 26.2, 27, 28, 29],\n",
    "        'RH (%)': [85, 84, 83, 82, 81],\n",
    "        'WS (m/s)': [0.7, 0.8, 0.9, 1.0, 1.1],\n",
    "        'WSgust (m/s)': [1.2, 1.4, 1.6, 1.8, 2.0],\n",
    "        'WSstdev (m/s)': [0.1, 0.1, 0.1, 0.1, 0.1],\n",
    "        'WD (°N (to east))': [210, 220, 230, 240, 250],\n",
    "        'WDstdev': [11, 12, 13, 14, 15],\n",
    "        'BP (hPa)': [995, 995, 996, 996, 997],\n",
    "        'Cleaning (1 or 0)': [0, 0, 0, 0, 1],\n",
    "        'Precipitation (mm/min)': [0, 0, 0, 0, 0],\n",
    "        'TModA (°C)': [26, 26, 27, 28, 29],\n",
    "        'TModB (°C)': [26, 26, 27, 28, 29],\n",
    "        'Comments': [np.nan, np.nan, np.nan, np.nan, 'Cleaning Event']\n",
    "    }\n",
    "    df_sierraleone = pd.DataFrame(data)\n",
    "\n",
    "print(\"First 5 rows of Sierra Leone data:\")\n",
    "df_sierraleone.head()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 2. Summary Statistics & Missing-Value Report"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": None,
   "metadata": {},
   "outputs": [],
   "source": [
    "print(\"--- Summary Statistics for Sierra Leone ---\")\n",
    "print(df_sierraleone.describe())\n",
    "\n",
    "print(\"\\n--- Missing Value Report for Sierra Leone ---\")\n",
    "missing_values = df_sierraleone.isna().sum()\n",
    "missing_percentage = (missing_values / len(df_sierraleone)) * 100\n",
    "\n",
    "missing_df = pd.DataFrame({\n",
    "    'Missing Count': missing_values,\n",
    "    'Missing %': missing_percentage\n",
    "})\n",
    "\n",
    "# Filter columns with missing values and sort\n",
    "missing_df = missing_df[missing_df['Missing Count'] > 0].sort_values(by='Missing %', ascending=False)\n",
    "print(missing_df)\n",
    "\n",
    "# List columns with >5% nulls\n",
    "cols_with_high_nulls = missing_df[missing_df['Missing %'] > 5].index.tolist()\n",
    "if cols_with_high_nulls:\n",
    "    print(f\"\\nColumns with more than 5% nulls: {cols_with_high_nulls}\")\n",
    "else:\n",
    "    print(\"\\nNo columns with more than 5% nulls.\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 3. Outlier Detection & Basic Cleaning"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": None,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Identify numeric columns for Z-score calculation\n",
    "numeric_cols_for_outliers = [\n",
    "    'GHI (W/m²)', 'DNI (W/m²)', 'DHI (W/m²)', 'ModA (W/m²)', 'ModB (W/m²)',\n",
    "    'WS (m/s)', 'WSgust (m/s)'\n",
    "]\n",
    "\n",
    "# Create a copy to avoid SettingWithCopyWarning\n",
    "df_sierraleone_cleaned = df_sierraleone.copy()\n",
    "\n",
    "# Handle missing values in numeric columns by imputing with median before Z-score calculation\n",
    "print(\"\\n--- Imputing Missing Values in Numeric Columns (Median) ---\")\n",
    "for col in numeric_cols_for_outliers:\n",
    "    if df_sierraleone_cleaned[col].isnull().any():\n",
    "        median_val = df_sierraleone_cleaned[col].median()\n",
    "        df_sierraleone_cleaned[col] = df_sierraleone_cleaned[col].fillna(median_val)\n",
    "        print(f\"Imputed missing values in '{col}' with median: {median_val}\")\n",
    "\n",
    "# Compute Z-scores and flag rows with |Z| > 3\n",
    "print(\"\\n--- Outlier Detection (|Z-score| > 3) ---\")\n",
    "for col in numeric_cols_for_outliers:\n",
    "    df_sierraleone_cleaned[f'Z_{col}'] = np.abs(zscore(df_sierraleone_cleaned[col]))\n",
    "    outliers = df_sierraleone_cleaned[df_sierraleone_cleaned[f'Z_{col}'] > 3]\n",
    "    if not outliers.empty:\n",
    "        print(f\"\\nPotential outliers flagged in {col} (rows where |Z| > 3):\")\n",
    "        print(outliers[['Timestamp', col, f'Z_{col}']].head())\n",
    "    else:\n",
    "        print(f\"\\nNo significant outliers found in {col} (|Z| > 3).\")\n",
    "\n",
    "# Basic cleaning: Fill 'Comments' column NaNs\n",
    "if 'Comments' in df_sierraleone_cleaned.columns:\n",
    "    initial_nan_count = df_sierraleone_cleaned['Comments'].isna().sum()\n",
    "    df_sierraleone_cleaned['Comments'] = df_sierraleone_cleaned['Comments'].fillna('No Comments')\n",
    "    if initial_nan_count > 0:\n",
    "        print(f\"\\nFilled {initial_nan_count} NaN values in 'Comments' column with 'No Comments'.\")\n",
    "    else:\n",
    "        print(\"\\nNo NaN values found in 'Comments' column.\")\n",
    "\n",
    "print(\"\\nSierra Leone data after basic cleaning and outlier flagging (first 5 rows):\")\n",
    "print(df_sierraleone_cleaned.head())\n",
    "\n",
    "# Export cleaned DataFrame\n",
    "output_path = '../data/sierraleone_clean.csv'\n",
    "df_sierraleone_cleaned.to_csv(output_path, index=False)\n",
    "print(f\"\\nCleaned Sierra Leone data exported to {output_path}\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 4. Time Series Analysis"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": None,
   "metadata": {},
   "outputs": [],
   "source": [
    "plt.figure(figsize=(15, 12))\n",
    "\n",
    "# GHI, DNI, DHI vs. Timestamp\n",
    "plt.subplot(4, 1, 1)\n",
    "plt.plot(df_sierraleone_cleaned['Timestamp'], df_sierraleone_cleaned['GHI (W/m²)'], label='GHI', color='darkorange')\n",
    "plt.plot(df_sierraleone_cleaned['Timestamp'], df_sierraleone_cleaned['DNI (W/m²)'], label='DNI', color='red', linestyle='--')\n",
    "plt.plot(df_sierraleone_cleaned['Timestamp'], df_sierraleone_cleaned['DHI (W/m²)'], label='DHI', color='green', linestyle=':')\n",
    "plt.title('Solar Irradiance Over Time (Sierra Leone)')\n",
    "plt.ylabel('Irradiance (W/m²)')\n",
    "plt.legend()\n",
    "plt.grid(True)\n",
    "\n",
    "# Tamb vs. Timestamp\n",
    "plt.subplot(4, 1, 2)\n",
    "plt.plot(df_sierraleone_cleaned['Timestamp'], df_sierraleone_cleaned['Tamb (°C)'], color='teal', label='Ambient Temperature')\n",
    "plt.title('Ambient Temperature Over Time (Sierra Leone)')\n",
    "plt.ylabel('Temperature (°C)')\n",
    "plt.legend()\n",
    "plt.grid(True)\n",
    "\n",
    "# Observe patterns by month (example: average GHI by month)\n",
    "df_sierraleone_cleaned['Month'] = df_sierraleone_cleaned['Timestamp'].dt.month\n",
    "monthly_ghi = df_sierraleone_cleaned.groupby('Month')['GHI (W/m²)'].mean()\n",
    "plt.subplot(4, 1, 3)\n",
    "monthly_ghi.plot(kind='bar', color='skyblue')\n",
    "plt.title('Average GHI by Month (Sierra Leone)')\n",
    "plt.xlabel('Month')\n",
    "plt.ylabel('Average GHI (W/m²)')\n",
    "plt.xticks(rotation=45)\n",
    "plt.grid(axis='y')\n",
    "\n",
    "# Observe patterns throughout the day (example: average GHI by hour)\n",
    "df_sierraleone_cleaned['Hour'] = df_sierraleone_cleaned['Timestamp'].dt.hour\n",
    "hourly_ghi = df_sierraleone_cleaned.groupby('Hour')['GHI (W/m²)'].mean()\n",
    "plt.subplot(4, 1, 4)\n",
    "hourly_ghi.plot(kind='line', marker='o', color='purple')\n",
    "plt.title('Average GHI by Hour of Day (Sierra Leone)')\n",
    "plt.xlabel('Hour of Day')\n",
    "plt.ylabel('Average GHI (W/m²)')\n",
    "plt.xticks(range(0, 24))\n",
    "plt.grid(True)\n",
    "\n",
    "plt.tight_layout()\n",
    "plt.show()\n",
    "\n",
    "print(\"Time series analysis plots generated.\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 5. Cleaning Impact"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": None,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Convert Cleaning column to category for better plotting\n",
    "df_sierraleone_cleaned['Cleaning (1 or 0)'] = df_sierraleone_cleaned['Cleaning (1 or 0)'].astype('category')\n",
    "\n",
    "plt.figure(figsize=(12, 6))\n",
    "sns.barplot(x='Cleaning (1 or 0)', y='ModA (W/m²)', data=df_sierraleone_cleaned, estimator=np.mean, errorbar='sd', palette='viridis')\n",
    "plt.title('Average ModA (W/m²) Pre/Post Cleaning (Sierra Leone)')\n",
    "plt.xlabel('Cleaning Event (0=No, 1=Yes)')\n",
    "plt.ylabel('Average ModA (W/m²)')\n",
    "plt.show()\n",
    "\n",
    "plt.figure(figsize=(12, 6))\n",
    "sns.barplot(x='Cleaning (1 or 0)', y='ModB (W/m²)', data=df_sierraleone_cleaned, estimator=np.mean, errorbar='sd', palette='magma')\n",
    "plt.title('Average ModB (W/m²) Pre/Post Cleaning (Sierra Leone)')\n",
    "plt.xlabel('Cleaning Event (0=No, 1=Yes)')\n",
    "plt.ylabel('Average ModB (W/m²)')\n",
    "plt.show()\n",
    "\n",
    "print(\"Cleaning impact plots generated.\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 6. Correlation & Relationship Analysis"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": None,
   "metadata": {},
   "outputs": [],
   "source": [
    "correlation_cols = [\n",
    "    'GHI (W/m²)', 'DNI (W/m²)', 'DHI (W/m²)', 'TModA (°C)', 'TModB (°C)',\n",
    "    'Tamb (°C)', 'RH (%)', 'WS (m/s)', 'BP (hPa)', 'Precipitation (mm/min)'\n",
    "]\n",
    "corr_matrix = df_sierraleone_cleaned[correlation_cols].corr()\n",
    "\n",
    "plt.figure(figsize=(12, 10))\n",
    "sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt=\".2f\", linewidths=.5)\n",
    "plt.title('Correlation Heatmap (Sierra Leone)')\n",
    "plt.show()\n",
    "\n",
    "# Scatter plots\n",
    "plt.figure(figsize=(18, 6))\n",
    "\n",
    "plt.subplot(1, 3, 1)\n",
    "sns.scatterplot(x='WS (m/s)', y='GHI (W/m²)', data=df_sierraleone_cleaned, alpha=0.6, color='darkblue')\n",
    "plt.title('Wind Speed vs. GHI (Sierra Leone)')\n",
    "\n",
    "plt.subplot(1, 3, 2)\n",
    "sns.scatterplot(x='WSgust (m/s)', y='GHI (W/m²)', data=df_sierraleone_cleaned, alpha=0.6, color='darkgreen')\n",
    "plt.title('Wind Gust vs. GHI (Sierra Leone)')\n",
    "\n",
    "plt.subplot(1, 3, 3)\n",
    "sns.scatterplot(x='RH (%)', y='Tamb (°C)', data=df_sierraleone_cleaned, alpha=0.6, color='darkred')\n",
    "plt.title('Relative Humidity vs. Ambient Temperature (Sierra Leone)')\n",
    "\n",
    "plt.tight_layout()\n",
    "plt.show()\n",
    "\n",
    "print(\"Correlation and relationship analysis plots generated.\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 7. Wind & Distribution Analysis"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": None,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Histograms\n",
    "plt.figure(figsize=(14, 6))\n",
    "\n",
    "plt.subplot(1, 2, 1)\n",
    "sns.histplot(df_sierraleone_cleaned['GHI (W/m²)'], bins=50, kde=True, color='purple')\n",
    "plt.title('Distribution of GHI (Sierra Leone)')\n",
    "plt.xlabel('GHI (W/m²)')\n",
    "plt.ylabel('Frequency')\n",
    "\n",
    "plt.subplot(1, 2, 2)\n",
    "sns.histplot(df_sierraleone_cleaned['WS (m/s)'], bins=50, kde=True, color='green')\n",
    "plt.title('Distribution of Wind Speed (Sierra Leone)')\n",
    "plt.xlabel('Wind Speed (m/s)')\n",
    "plt.ylabel('Frequency')\n",
    "\n",
    "plt.tight_layout()\n",
    "plt.show()\n",
    "\n",
    "# Wind rose / Radial Bar Plot for Wind Direction and Speed\n",
    "# Group wind speeds by wind direction bins\n",
    "bins = np.arange(0, 361, 30) # 30-degree bins for finer detail\n",
    "df_sierraleone_cleaned['WD_Bin'] = pd.cut(df_sierraleone_cleaned['WD (°N (to east))'], bins, right=False, labels=bins[:-1])\n",
    "avg_ws_by_wd = df_sierraleone_cleaned.groupby('WD_Bin')['WS (m/s)'].mean().reindex(bins[:-1])\n",
    "avg_ws_by_wd = avg_ws_by_wd.fillna(0) # Fill NaN if a bin has no data\n",
    "\n",
    "plt.figure(figsize=(9, 9))\n",
    "ax = plt.subplot(111, polar=True)\n",
    "ax.bar(np.radians(avg_ws_by_wd.index.astype(float)), avg_ws_by_wd, width=np.radians(30), bottom=0.0, color='blue', alpha=0.7)\n",
    "ax.set_theta_zero_location('N') # North at the top\n",
    "ax.set_theta_direction(-1) # Clockwise direction\n",
    "ax.set_title('Average Wind Speed by Wind Direction (Sierra Leone)', va='bottom', fontsize=14)\n",
    "\n",
    "# Set labels for cardinal directions\n",
    "ax.set_xticks(np.radians([0, 45, 90, 135, 180, 225, 270, 315]))\n",
    "ax.set_xticklabels(['N', 'NE', 'E', 'SE', 'S', 'SW', 'W', 'NW'])\n",
    "\n",
    "plt.show()\n",
    "\n",
    "print(\"Wind and distribution analysis plots generated.\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 8. Temperature Analysis (Influence of RH on Temp and GHI)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": None,
   "metadata": {},
   "outputs": [],
   "source": [
    "plt.figure(figsize=(12, 7))\n",
    "sns.scatterplot(x='RH (%)', y='Tamb (°C)', hue='GHI (W/m²)', size='GHI (W/m²)', sizes=(20, 600), \n",
    "                data=df_sierraleone_cleaned, palette='viridis', alpha=0.7)\n",
    "plt.title('Relative Humidity vs. Ambient Temperature, Colored and Sized by GHI (Sierra Leone)')\n",
    "plt.xlabel('Relative Humidity (%)')\n",
    "plt.ylabel('Ambient Temperature (°C)')\n",
    "plt.legend(title='GHI (W/m²)', bbox_to_anchor=(1.05, 1), loc='upper left')\n",
    "plt.grid(True, linestyle='--', alpha=0.6)\n",
    "plt.show()\n",
    "\n",
    "print(\"Temperature analysis plot generated.\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 9. Bubble Chart: GHI vs. Tamb with bubble size = RH"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": None,
   "metadata": {},
   "outputs": [],
   "source": [
    "plt.figure(figsize=(12, 8))\n",
    "sns.scatterplot(x='Tamb (°C)', y='GHI (W/m²)', size='RH (%)', hue='RH (%)', sizes=(50, 1000),\n",
    "                data=df_sierraleone_cleaned, palette='coolwarm', alpha=0.7)\n",
    "plt.title('GHI vs. Ambient Temperature (Sierra Leone), Bubble Size & Color by Relative Humidity')\n",
    "plt.xlabel('Ambient Temperature (°C)')\n",
    "plt.ylabel('GHI (W/m²)')\n",
    "plt.legend(title='RH (%)', bbox_to_anchor=(1.05, 1), loc='upper left')\n",
    "plt.grid(True, linestyle='--', alpha=0.6)\n",
    "plt.show()\n",
    "\n",
    "print(\"Bubble chart generated.\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": None,
   "metadata": {},
   "outputs": [],
   "source": []
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3 (ipykernel)",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.10.12"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 4
}

{'cells': [{'cell_type': 'markdown',
   'metadata': {},
   'source': ['# EDA for Sierra Leone Solar Farm Data']},
  {'cell_type': 'code',
   'execution_count': None,
   'metadata': {},
   'outputs': [],
   'source': ['import pandas as pd\n',
    'import numpy as np\n',
    'import matplotlib.pyplot as plt\n',
    'import seaborn as sns\n',
    'from scipy.stats import zscore\n',
    '\n',
    'sns.set_style("whitegrid")']},
  {'cell_type': 'markdown',
   'metadata': {},
   'source': ['## 1. Setup and Data Loading']},
  {'cell_type': 'code',
   'execution_count': None,
   'metadata': {},
   'outputs': [],
   'source': ['# Load the Sierra Leone dataset\n',
    "# Ensure the 'data' directory is created and 'sierraleone-bumbuna.csv' is placed inside it.\n",
    'try:\n',
    "    df_sierraleone = pd.read_csv('../data/sierraleone-bumbuna.csv', parse_dates=['Timestamp (yyyy-mm-dd hh:mm)'])\n",
    "    df_sierraleone = df_sierraleone.rename(columns={'Timestamp (yyyy-mm-dd hh:mm)': 'Timestamp